In [1]:
#import libraries

import openai
import langchain
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.llms import OpenAI

e:\Chat-Bot\OpenAi-ChatBot\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [4]:
#let's read the document
def read_doc (directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [5]:
docs = read_doc('documents/')
len(docs)

143

In [6]:
## Divide the docs into chunks

def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size , chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc

In [7]:
documents = chunk_data(docs=docs)
len(documents)

700

In [8]:
## Embedding technique of OPENAI

embeddings = OpenAIEmbeddings(api_key = os.environ['API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000026B55F62B70>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000026B55F62990>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [9]:
## Vector Search DB in pinecone
pc = Pinecone(api_key = os.environ['LANG_KEY'], region= 'us-east-1')
index_name = "langchainvector"



In [10]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

In [11]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'langchainvector-nbb3l4m.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchainvector',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [12]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings, pinecone_api_key=os.environ['LANG_KEY'])


In [13]:
vectorstore_from_docs = vectorstore.add_documents(documents=docs)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
##Cosine similarity results from vectorDB       
def retrieve_query(query, k=2):
    matching_results = vectorstore_from_docs.similarity_search(query , k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm = OpenAI(api_key = os.environ['API_KEY'],model_name="text-davinci-003", temparture=0.5)
chain = load_qa_chain(llm=llm, chain_type="stuff")

In [ ]:
## Search answers from vectorDB

def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents = doc_search, question=query)
    return response

In [ ]:
our_query = "Whats the pdf is about"
answer = retrieve_answers(our_query)
print(answer)